In [217]:
import numpy as np
import csv
import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from tokenize import TokenError
import heapq

In [218]:
# function to create default policy values. Unimplemented policies are NaN, rest are 0.5
def set_policy_defaults(policy_idxs):
    available_policies = ["RailSubsidies", "RoadBuilding", "BusLanes", "BorderControls", "MilitarySpending",
                          "ForeignAid", "ChildBenefit", "StatePensions", "StateHousing", "UnemployedBenefit",
                          "CleanEnergySubsidies", "Recycling", "LabourLaws", "AgricultureSubsidies",
                          "PollutionControls", "TobaccoTax", "IncomeTax", "PetrolTax", "AlcoholTax", "InheritanceTax",
                          "SalesTax", "CapitalGainsTax", "CorporationTax", "PropertyTax", "ScienceFunding",
                          "AbortionLaw", "StateHealthService", "FoodStandards", "Creationism", "StateSchools",
                          "JuryTrial", "LegalAid", "CCTVCameras", "Narcotics", "IntelligenceServices", "AlcoholLaw",
                          "HandgunLaws", "PoliceForce", "CommunityPolicing", "RaceDiscriminationAct", "Prisons"]
    
    policy_defaults = [sp.nan]*len(policy_idxs)
    
    for policy in available_policies:
        policy_defaults[policy_idxs[policy]] = 0.5
    
    return policy_defaults

In [219]:
# Gathering all variable information and interdependency
variable_idxs = {}
variable_names = []
variable_defaults = []
variable_mins = []
variable_maxs = []
# NxN matrix where matrix[i, j] = effect of variable i on variable j
variable_to_variable_effects = []
# NxN matrix where matrix[i, j] = delay of change in variable j from a change in variable i
variable_to_variable_delay = []

with open("/Users/shrikar.amirisetty/Library/Application Support/Steam/steamapps/common/Democracy 3/Democracy3.app/Contents/Resources/data/simulation/simulation.csv", newline="") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader)
  
    # Taking values from csv rows
    idx = 0
    for row in csvreader:
        variable_idxs[row[1]] = idx
        variable_names.append(row[2])
        variable_defaults.append(float(row[5]))
        variable_mins.append(int(row[6]))
        variable_maxs.append(int(row[7]))
        idx += 1
    
    # Adding 'year' variable to arrays
    variable_idxs['_year'] = idx
    variable_names.append("Year")
    variable_defaults.append(0)
    variable_mins.append(0)
    variable_maxs.append(np.inf)
    
    # Initialize effects and delay matrix and added logic for 'year'
    
    variable_to_variable_effects = sp.zeros(len(variable_idxs), len(variable_idxs))
    variable_to_variable_delay = sp.zeros(len(variable_idxs), len(variable_idxs))
    
    x = sp.Symbol('x')
    variable_to_variable_effects[-1, -1] = x + 0.25
    
    # Taking effects expressions and delays from csv rows
    csvfile.seek(0)
    next(csvreader)
    for row in csvreader:
        for i in range(11, len(row)):
            if row[i] == '':
                break
            if row[i] == '#':
                continue
            effect_string_split = row[i].split(',')
            if effect_string_split[0] not in variable_idxs:
                continue
            variable_to_variable_effects[variable_idxs[effect_string_split[0]], variable_idxs[row[1]]] = parse_expr(effect_string_split[1].replace('^', '**'))
            if len(effect_string_split) >= 3:
                variable_to_variable_delay[variable_idxs[effect_string_split[0]], variable_idxs[row[1]]] = int(effect_string_split[2])
    
# Gathering all policies information and interdependency
policy_idxs = {}
policy_names = []
policy_defaults = []
# NxN matrix where matrix[i, j] = effect of poliy i on variable j
policy_to_variable_effects = []
# NxN matrix where matrix[i, j] = delay of change in variable j from a change in policy i
policy_to_variable_delay = []

with open("/Users/shrikar.amirisetty/Library/Application Support/Steam/steamapps/common/Democracy 3/Democracy3.app/Contents/Resources/data/simulation/policies.csv", newline="") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    next(csvreader)
  
    # Taking values from csv rows
    idx = 0
    for row in csvreader:
        policy_idxs[row[1]] = idx
        policy_names.append(row[2])
        idx += 1
    
    policy_defaults = set_policy_defaults(policy_idxs)
    
    policy_to_variable_effects = sp.zeros(len(policy_idxs), len(variable_idxs))
    policy_to_variable_delay = sp.zeros(len(policy_idxs), len(variable_idxs))
    
    # Taking effects expressions and delays from csv rows
    csvfile.seek(0)
    next(csvreader)
    for row in csvreader:
        for i in range(19, len(row)):
            if row[i] == '' or row[i] == ' ':
                break
            effect_string_split = row[i].split(',')
            if effect_string_split[0] not in variable_idxs:
                continue
            try:
                policy_to_variable_effects[policy_idxs[row[1]], variable_idxs[effect_string_split[0]]] = parse_expr(effect_string_split[1].replace('^', '**').replace('0.0.5', '0.0'))
            except TokenError:
                effect_string_split[1] = effect_string_split[1][:-1]
                policy_to_variable_effects[policy_idxs[row[1]], variable_idxs[effect_string_split[0]]] = parse_expr(effect_string_split[1].replace('^', '**').replace('0.0.5', '0.0'))
            if len(effect_string_split) >= 3:
                policy_to_variable_delay[policy_idxs[row[1]], variable_idxs[effect_string_split[0]]] = int(effect_string_split[2])

# Generating and assigning symbols to represent each variable and policy value
symbols_list = list(variable_idxs.keys()) + list(policy_idxs.keys())
var = sp.symbols(" ".join(symbols_list))

variable_symbols = {}
policy_symbols = {}

for variable in variable_idxs:
    variable_to_variable_effects[variable_idxs[variable], :] = variable_to_variable_effects[variable_idxs[variable], :].subs(x, var[variable_idxs[variable]])
    variable_symbols[variable] = var[variable_idxs[variable]]

for policy in policy_idxs:
    policy_to_variable_effects[policy_idxs[policy], :] = policy_to_variable_effects[policy_idxs[policy], :].subs(x, var[policy_idxs[policy]+len(variable_idxs)])
    policy_symbols[policy] = var[policy_idxs[policy]+len(variable_idxs)]
        

In [235]:
def no_change_iter(epochs, heap, curr_vals_variables, variable_to_variable_effects_populated, policy_to_variable_effects_populated):
    for epoch in range(epochs):
        for i in range(len(variable_idxs)):
            variable_to_variable_effects_populated[i, :] = variable_to_variable_effects[i, :].subs(var[i], curr_vals_variables[i])
        for variable in ["CarUsage", "TobaccoUse", "GDP"]:
            policy_to_variable_effects_populated = policy_to_variable_effects_populated.subs(variable_symbols[variable], curr_vals_variables[variable_idxs[variable]])
        for i in range(len(variable_idxs)):
            curr_vals_variables[i] = variable_defaults[i] + sum(variable_to_variable_effects_populated[:, i]) + sum(policy_to_variable_effects_populated[:, i])
            curr_vals_variables[i] = max(min(curr_vals_variables[i], variable_maxs[i]), variable_mins[i])

def change_iter(epochs, heap, policy_to_change, new_value, curr_vals_variables, variable_to_variable_effects_populated, policy_to_variable_effects_populated):
    
            
def simul(epochs, policy_to_change, new_value):
    curr_vals_variables = sp.Matrix(variable_defaults)
    curr_vals_policies = sp.Matrix(policy_defaults)
    
    variable_to_variable_effects_populated = variable_to_variable_effects.copy()
    policy_to_variable_effects_populated = policy_to_variable_effects.copy()

    for i in range(len(policy_idxs)):
        if curr_vals_policies[i] == sp.nan or i == policy_idxs[policy_to_change]:
            policy_to_variable_effects_populated[i, :] *= 0
        else:
            policy_to_variable_effects_populated[i, :] = policy_to_variable_effects[i, :].subs(var[i+len(variable_idxs)], curr_vals_policies[i])
    
    heap = []
    for i in range(len(variable_idxs)):
        for j in range(len(variable_idxs)):
            
    no_change_iter(15, heap, curr_vals_variables, variable_to_variable_effects_populated, policy_to_variable_effects_populated)
    
    curr_vals_variables_test = curr_vals_variables.copy()
    variable_to_variable_effects_populated_test = variable_to_variable_effects_populated.copy()
    policy_to_variable_effects_populated_test = policy_to_variable_effects_populated.copy()
    
    no_change_iter(epochs, heap, curr_vals_variables_test, variable_to_variable_effects_populated_test, policy_to_variable_effects_populated_test)
    
    change_iter(epochs, heap, policy_to_change, new_value, curr_vals_variables, variable_to_variable_effects_populated, policy_to_variable_effects_populated)
    
    return curr_vals_variables

In [236]:
vals = simul("RailSubsidies", 0)

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9


In [237]:
vals

Matrix([
[    0.6438218078385],
[  0.434123302221439],
[   0.13565078181231],
[                  0],
[  0.530013566789694],
[  0.985218002138253],
[                0.9],
[  0.582905339137383],
[  0.481549642922704],
[  0.863789196833403],
[ 0.0195932160192862],
[ 0.0901993794793507],
[  0.463872257184073],
[  0.209018098050131],
[                  0],
[                  0],
[  0.586201639248507],
[  0.626061537159203],
[                  0],
[  0.577247082075105],
[  0.495034850463001],
[  0.353859818148708],
[  0.212390181851292],
[  0.509109957591992],
[             0.4225],
[  0.165494812777011],
[-0.0200673271472839],
[ -0.164977782625275],
[  0.524247699131323],
[  0.904663777964146],
[  0.638342339694484],
[  0.863052031165836],
[  0.568424030873446],
[  0.272254118666328],
[  0.404216280167957],
[  0.878220343929076],
[  0.621631910087058],
[                  0],
[                  0],
[  0.499284074447182],
[                2.5]])